In [6]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import classification, Object,storing
import random

In [7]:
def MLAData(Full_Data,BinInfos,Flux,log_wavs,ANDMASK, INV, MJDs, PlateIDs):
    
    
    All_Y=[]
    All_X = []
    All_redshifts=[]
    All_Mag=[]
    All_AND=[]
    All_Inv=[]
    All_Name=[]
    All_MJDs = []
    MatchedPlates = []
    wav_logs=[]
    plate_no = 0
    y=0
    while plate_no < len(Full_Data):
        #loading matched objects with sup, plate data
        CurrentSup_data = Full_Data[plate_no]
        CurrentBin = BinInfos[plate_no]
        CurrentFlux = Flux[plate_no]
        CurrentAndM= ANDMASK[plate_no]
        CurrentInv = INV[plate_no]
        wav= log_wavs[plate_no]
        currentmjd = MJDs[plate_no]
        Plate_Y = []
        Plate_X = []
        Plate_AND=[]
        Plate_Inv=[]
        Plate_redshifts=[]
        Plate_Mag=[]
        Plate_Name=[]
        pid=PlateIDs[plate_no]

        plate_match = False
        
        #first object is zeroth element
        Sup_obj =0 
        while Sup_obj < len(CurrentSup_data):
            #to stop double counting
            no_match = True
            #looking at an object that has been matched in sup list
            CurrentSup = CurrentSup_data[Sup_obj]            
            #going through each object in the bin
            BinObj_No = 0
            while BinObj_No<len(CurrentBin):
                BinObj = CurrentBin[BinObj_No]
                ObjAndM= CurrentAndM[BinObj_No]
                ObjInv = CurrentInv[BinObj_No]
                
                ##checking if the two match 
                if BinObj['FIBERID'] == CurrentSup.FiberID:
                    if BinObj['FIBERID'] == CurrentSup.FiberID:
                        plate_match = True 
                        if no_match:
                            if CurrentSup.Class_p == 0:
                                a=0
                            else:
                                no_match = False
                                if CurrentSup.Class_p == 3 or CurrentSup.Class_p==30:
                                    if CurrentSup.z < 2.1:
                                        Plate_Y.append(1)
                                        x_flux =(CurrentFlux[BinObj_No])
                                        x_flux=x_flux[:4600]
                                        Plate_X.append(x_flux)
                                        Plate_redshifts.append(CurrentSup.z)
                                        Plate_AND.append(ObjAndM)
                                        Plate_Inv.append(ObjInv)
                                        Plate_Mag.append(CurrentSup.Mag)
                                        Plate_Name.append(CurrentSup.name)
                                       
                                        
                                    else:
                                        Plate_Y.append(3)
                                        x_flux =(CurrentFlux[BinObj_No])
                                        x_flux=x_flux[:4600]
                                        Plate_X.append(x_flux)
                                        Plate_redshifts.append(CurrentSup.z)
                                        Plate_AND.append(ObjAndM)
                                        Plate_Inv.append(ObjInv)
                                        Plate_Mag.append(CurrentSup.Mag)
                                        Plate_Name.append(CurrentSup.name)
                                        
                                                
                                    
                                else: 
                                    if CurrentSup.Class_p ==1:
                                        Plate_Y.append(0)
                                        x_flux =(CurrentFlux[BinObj_No])
                                        x_flux=x_flux[:4600]
                                        Plate_X.append(x_flux)
                                        Plate_redshifts.append(CurrentSup.z)
                                        Plate_AND.append(ObjAndM)
                                        Plate_Inv.append(ObjInv)
                                        Plate_Mag.append(CurrentSup.Mag)
                                        Plate_Name.append(CurrentSup.name)
                                        
                                    else:
                                        Plate_Y.append(2)
                                        x_flux =(CurrentFlux[BinObj_No])
                                        x_flux=x_flux[:4600]
                                        Plate_X.append(x_flux)
                                        Plate_redshifts.append(CurrentSup.z)
                                        Plate_AND.append(ObjAndM)
                                        Plate_Inv.append(ObjInv)
                                        Plate_Mag.append(CurrentSup.Mag)
                                        Plate_Name.append(CurrentSup.name)
                                    
                                        
            
                BinObj_No=BinObj_No+1  
            Sup_obj=Sup_obj+1
        if plate_match:
            All_Y.append(Plate_Y)
            All_X.append(Plate_X)
            All_Name.append(Plate_Name)
            plate_no = plate_no+1
            wav_logs.append(wav)
            All_redshifts.append(Plate_redshifts)
            All_Mag.append(Plate_Mag)
            All_AND.append(Plate_AND)
            All_Inv.append(Plate_Inv)
            All_MJDs.append(currentmjd)
            MatchedPlates.append(pid)
            plate_no=plate_no+1
        else:
            plate_no=plate_no+1
    

    return All_X,All_Y,All_redshifts,All_Mag,All_AND,All_Inv,wav_logs,All_Name,All_MJDs,MatchedPlates

In [8]:
TrainPlateDir = os.path.normpath("D:\Data\Plate_Name.txt")
with open(TrainPlateDir) as f:
    Spectra_Files = f.read().splitlines()


In [9]:
PLATEIDs = []
BinInfos = []
Flux = []
MJDs = []
log_wavst=[]
ANDMASK=[]
INVAR=[]
TrainingDir = os.path.normpath("D:\Data")
TrainingFolder =  os.path.normpath("\Training")
slash =  os.path.normpath("\\")
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir +TrainingFolder+slash+ spectrum ,memmap=True)
    Bin_info_ = plate_[5].data
    Flux_ = plate_[0].data
    primhdu_ = plate_[0]
    ANDMASK.append( plate_[2].data)
    INVAR.append( plate_[1].data)
    PLATEIDs.append(primhdu_.header['PLATEID'])
    log_wavst.append(primhdu_.header['COEFF0'])
    MJDs.append(primhdu_.header['MJD'])
    BinInfos.append(Bin_info_)
    Flux.append(Flux_)
    

In [16]:
list = fits.open(TrainingDir+slash+'Superset_DR12Q.fits',memmap=True)#opening file

supers=list[1].data # storing  BINTABLE extension data

In [18]:
Full_Data = storing(PLATEIDs,supers)

In [20]:
len(BinInfos)

82

In [22]:
X,Y,Train_z, Train_mag,All_AND,All_Inv,wav_logs,All_Name,All_MJDs,MatchedPlates= MLAData(Full_Data,BinInfos,Flux, log_wavst,ANDMASK,INVAR,MJDs,PLATEIDs)

In [24]:
len(X)

41

In [26]:
TestPlateDir = os.path.normpath("D:\Data\TestPlate_Name.txt")
with open(TestPlateDir) as f:
    Spectra_TestFiles = f.read().splitlines()
Spectra_TestFiles

['spPlate-4791-55889.fits',
 'spPlate-6970-56444.fits',
 'spPlate-5852-56034.fits',
 'spPlate-5484-56039.fits',
 'spPlate-6290-56238.fits',
 'spPlate-6122-56246.fits',
 'spPlate-6424-56272.fits',
 'spPlate-6723-56428.fits',
 'spPlate-6880-56543.fits']

In [28]:
TrainingDir = os.path.normpath("D:\Data")
TestingFolder =  os.path.normpath("\Test")
PLATEIDs_test = []
log_wavs=[]
BinInfos_test = []
Flux_test = []
AT = []
IT = []
MJDs_Test=[]
z=0
for spectrum_test in Spectra_TestFiles:
    plate_test = fits.open(TrainingDir +TestingFolder+slash+spectrum_test ,memmap=True)
    Bin_info_test = plate_test[5].data
    Flux_t = plate_test[0].data
    primhdu_test = plate_test[0]
    AT.append( plate_[2].data)
    IT.append( plate_[1].data)
    PLATEIDs_test.append(primhdu_test.header['PLATEID'])
    log_wavs.append(primhdu_test.header['COEFF0'])
    MJDs_Test.append(primhdu_.header['MJD'])
    BinInfos_test.append(Bin_info_test)
    Flux_test.append(Flux_t)


In [30]:
Full_Data_test = storing(PLATEIDs_test,supers)
print(len(Full_Data_test))

9


In [32]:
print(len(Full_Data_test[1]))

302


In [34]:
X_test,Y_test,All_redshifts,All_Mag,All_AND,All_Inv,wav_logs,All_Name,g,g4= MLAData(Full_Data_test,BinInfos_test,Flux_test,log_wavs,AT,IT,MJDs_Test,PLATEIDs_test )
print(len(X_test))

5


Solvers: ‘lbfgs’ is an optimizer in the family of quasi-Newton methods. ‘sgd’ refers to stochastic gradient descent. ‘adam’ refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba

In [36]:
hiddenlayer_format = (13)
backprop_method = 'lbfgs'
lr=0.00001
act =  'tanh' #'logistic'
mlp = MLPClassifier(hidden_layer_sizes=hiddenlayer_format,max_iter=500, solver = backprop_method,learning_rate_init=lr,activation=act) ##Think About

In [38]:
X_Full=[]
Y_Full = []
n=0
total = 0
while n< len(X):
    currentbinX = X[n]
    currentbinY = Y[n]
    i=0
    total=total + len(currentbinY)
    while i < len(currentbinY):
        X_Full.append(currentbinX[i])
        Y_Full.append(currentbinY[i])
        i=i+1
    n=n+1
        


print(len(X_Full))
print(total)

mlp.fit(X_Full,Y_Full)

8988
8988


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=13, learning_rate='constant',
       learning_rate_init=1e-05, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [40]:
XT_Full=[]
YT_Full = []
n=0
print(len(X_test))
while n< len(X_test):
    currentbinX = X_test[n]
    currentbinY = Y_test[n]
    i=0
    total=total + len(currentbinY)
    while i < len(currentbinY):
        XT_Full.append(currentbinX[i])
        YT_Full.append(currentbinY[i])
        i=i+1
    n=n+1
print(len(XT_Full))

predictions = mlp.predict(np.array(XT_Full))

5
1083


In [42]:
print(classification_report(YT_Full,predictions))

             precision    recall  f1-score   support

          0       0.86      0.88      0.87       408
          1       0.76      0.67      0.71       213
          2       0.21      0.31      0.25        26
          3       0.89      0.89      0.89       436

avg / total       0.84      0.83      0.83      1083



In [ ]:
star,star_starloc,star_lowzloc,star_galloc,star_highzloc = classification(0,YT_Full,predictions) 
lowz,lowz_starloc,lowz_loc,lowz_galloc,lowz_highzloc = classification(1,YT_Full,predictions)
gal,gal_starloc,gal_lowzloc,gal_galloc,gal_highzloc = classification(2,YT_Full,predictions)
highz,highz_starloc,highz_lowzloc,highz_galloc,highz_highzloc = classification(3,YT_Full,predictions)

In [ ]:
File_Name = input("Please Enter File name: ")

In [ ]:
d = open(TrainingDir+slash+File_Name+".txt", 'w')

In [ ]:
t1=["Files used to train: ",np.str(Spectra_Files), "\n"]
sp= "\n"
t2 = ["Files used to test: ",np.str(Spectra_TestFiles), "\n"]
t3 = ["Number of training objects = ",np.str(len(Spectra_Files)), "\n"]
t4 = ["Number of testing objects = ",np.str(len(Spectra_TestFiles)), "\n"]
n1 = ["Structure of neural network: ", np.str(hiddenlayer_format),"\n"]
n2  = ["Backpropagation method used: ",np.str(backprop_method), "\n"]
n3  = ["Learning rate: ",np.str(lr), "\n"]
n4  = ["Activation Function: ",np.str(act), "\n"]
r1 = ["Results of Neural Network: ", "\n","\n"]
r2=["        ","       Star    Quasar  Galaxy  BAL ","\n",]
r3="Star           ",np.str(np.round(star[0]*100,2)),"%  ", np.str(np.round(star[1]*100,2)),"%  ",np.str(np.round(star[2]*100,2)),"%  ",np.str(np.round(star[3]*100,2)),"%","\n"
r4="Quasar z<2.1   ",np.str(np.round( lowz[0]*100,2)),"%  ", np.str(np.round( lowz[1]*100,2)),"%  ",np.str(np.round( lowz[2]*100,2)),"%  ",np.str(np.round( lowz[3]*100,2)),"%","\n"
r5="Galaxy         ",np.str(np.round( gal[0]*100,2)),"%  ", np.str(np.round( gal[1]*100,2)),"%  ",np.str(np.round( gal[2]*100,2)),"%  ",np.str(np.round( gal[3]*100,2)),"%","\n"
r6="Quasar z<2.1   ",np.str(np.round( highz[0]*100,2)),"%  ", np.str(np.round( highz[1]*100,2)),"%  ",np.str(np.round( highz[2]*100,2)),"%  ",np.str(np.round (highz[3]*100,2)),"%","\n"

In [ ]:
d.writelines(t1)
d.writelines(sp)
d.writelines(t2)
d.writelines(sp)
d.writelines(t3)
d.writelines(t4)
d.writelines(sp)
d.writelines(sp)
d.writelines(n1)
d.writelines(n2)
d.writelines(n3)
d.writelines(n4)
d.writelines(sp)
d.writelines(sp)
d.writelines(r1)
d.writelines(r2)
d.writelines(r3)
d.writelines(r4)
d.writelines(r5)
d.writelines(r6)


In [ ]:
list.close()

In [ ]:
d.close()

In [44]:
misclass_highz1=random.choice(highz_starloc)
#misclass_highz2=random.choice(highz_galloc)
misclass_star=random.choice(star_lowzloc)
misclass_gal= random.choice(gal_lowzloc)


From (http://www.sdss.org/dr12/spectro/spectro_basics/) : 'About the Spectra', we  know:


<h5><center>log($\lambda_{i+1}$) - log($\lambda_{i}$)=0.0001.</center></h5>

I will use this, and the first pixels central wavelength, to produce the wavelength array for this plate. This will be done by finding the plate wavelength ratio: 

<h3><center>$\frac{\lambda_{i+1}}{\lambda_{i}}= 10^{0.0001}$.</center></h3>


In [46]:
w_full=[]
i=0
while i<len(wav_logs):
    w_full.append(wav_logs[i]) 
    i=i+1
len(w_full)

5

In [48]:
wav_ratio = 10**0.0001
append_count=0
cent_wav = 10**wav_logs[0]
Y_flux = XT_Full[misclass_highz1]
wavelengths = []
wavelengths.append(cent_wav)
current_wav = cent_wav
while append_count < len(Y_flux)-1:
    current_wav = current_wav*wav_ratio
    wavelengths.append(current_wav)
    append_count=append_count+1
    
print(misclass_highz1)

776


In [50]:
wav_log=[]
Full_table = []
PLATEIDs = []
bin_size = np.str(10)
TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")
PlateDir ="D:"+slash+"Data"+slash+"FullRebin10"+".txt"
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir+slash+'rebintrain'+slash+spectrum ,memmap=True)
    primhdu_ = plate_[0]
    hdu =  plate_[1].data
    Full_table.append(hdu)
    PLATEIDs.append(primhdu_.header['Plate'])
    wav_log.append(primhdu_.header['LogWav'])

In [52]:
plate_r  = []
plate_n  = []
plate_c  = []
plate_x  = []
plate_y  = []
i = 0
while i < len(PLATEIDs):
    plate_hdu  = Full_table[i]
    obj = 0
    while obj <len(plate_hdu):
        currentobj = plate_hdu[obj]
        Currentplate_z = currentobj[4]
        Currentplate_y = currentobj[2]
        Currentplate_x = currentobj[1]
        Currentplate_n = currentobj[0]
        plate_r.append(Currentplate_z)
        plate_n.append(Currentplate_n)
        plate_x.append(Currentplate_x[:800])
        plate_y.append(Currentplate_y)
        obj = obj +1
    i = i+1



In [53]:
bin_wav = 10*0.829026074968624
bin_max = 3600
i=0
cwavbin=[]
while i<800:
    cwavbin.append(bin_max+(bin_wav*0.5))
    bin_max = bin_max+bin_wav
    i=i+1
    
normal_flux= plate_x[misclass_gal]

In [ ]:
plt.plot(wavelengths,Y_flux)
plt.ylabel('Flux [$10^{-17}$ $erg/s/cm^2/$'r'$\AA$'']')
plt.xlabel('Wavelength' r' $\AA$')
plt.title('Misclassified Quasar')
plt.show()

In [54]:
append_count=0
cent_wav2 = 10**wav_logs[0]
Y2_flux = XT_Full[misclass_star]
wavelengths2 = []
wavelengths2.append(cent_wav2)
current_wav2 = cent_wav2
while append_count < len(Y2_flux)-1:
    current_wav2 = current_wav2*wav_ratio
    wavelengths2.append(current_wav2)
    append_count=append_count+1

In [ ]:
plt.plot(wavelengths2,Y2_flux)
plt.ylabel('Flux [$10^{-17}$ $erg/s/cm^2/$'r'$\AA$'']')
plt.xlabel('Wavelength' r' $\AA$')
plt.title('Misclassified Star')
plt.show()

In [55]:
#append_count=0
#cent_wav3 = 10**wav_logs[0]
Y3_flux = XT_Full[misclass_gal]
#wavelengths3 = []
#wavelengths3.append(cent_wav)
#current_wav3 = cent_wav3
#while append_count < len(Y3_flux)-1:
#    current_wav3 = current_wav3*wav_ratio
 #   wavelengths3.append(current_wav3)
  #  append_count=append_count+1

In [56]:

plt.plot(wavelengths2,Y3_flux)
plt.plot(cwavbin,normal_flux)


plt.ylabel('Flux [$10^{-17}$ $erg/s/cm^2/$'r'$\AA$'']')
plt.xlabel('Wavelength' r' $\AA$')
plt.title('Misclassified Galaxy')
plt.savefig('galspecoverplot.png')

In [ ]:
append_count=0
cent_wav4 = 10**wav_logs[misclass_gal]
Y4_flux = X_test[misclass_gal]
wavelengths4 = []
wavelengths4.append(cent_wav4)
current_wav4 = cent_wav4
while append_count < len(Y4_flux)-1:
    current_wav4 = current_wav4*wav_ratio
    wavelengths4.append(current_wav4)
    append_count=append_count+1

In [ ]:
plt.plot(wavelengths4,Y4_flux)
plt.ylabel('Flux [$10^{-17}$ $erg/s/cm^2/$'r'$\AA$'']')
plt.xlabel('Wavelength' r' $\AA$')
plt.title('Misclassified Star as Quasar with z>2.1')
plt.show()

In [ ]:
Bin_no = input("Please Enter bin : ")

In [ ]:
X_bin,Y_bin,Train_z_bin, Train_mag_bin,wavst_bin = MLAData(Full_Data,BinInfos,Flux, log_wavst,Bin_no)

In [ ]:
X_test_bin,Y_test_bin,All_redshifts_bint,All_Ma_bintg,wav_logs_bin= MLAData(Full_Data_test,BinInfos_test,Flux_test,log_wavs)